In [5]:
# импорт библиотек
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [6]:
# загружаем датафрейм с анекдотами
df_original = pd.read_csv('posts.csv').set_index('id')
df_original

,text
id,
0,"Стоят два наркомана на перроне, мимо проезжает..."
1,В советские времена объявили месяц вежливого о...
2,Полицейские повязали целую группу проституток ...
3,"— Вы уверены, что хотите выключить компьютер? ..."
4,"Извините, но подписчик из меня так себе"
...,...
24651,Марк Цукерберг и Павел Дуров заходят в бар. Цу...
24652,"— Вы любите Кафку? — Да, грефневую!"
24653,На распродаже человеческих органов нaчалась да...


Некоторые матерные слова зацензурены символами * или # что усложняет обработку этих слов. Анекдотов содержащих данные символы немного, поэтому можно их и удалить. Также уберем все анекдоты содержащие английские символы.

In [7]:
df = df_original[~df_original.text.str.contains('[\*#\[\]a-zA-Z]')]
df.drop(df[df.text.str.len() < 70].index, inplace=True)
df

C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text
id,
0,"Стоят два наркомана на перроне, мимо проезжает..."
1,В советские времена объявили месяц вежливого о...
2,Полицейские повязали целую группу проституток ...
5,"— Дед, почему наше село называется Астафьево? ..."
8,"— Алло, здравствуйте, это Ремонт Ноутбуков? — ..."
...,...
24646,"Пожар в Техасе, в школе: - Кидай мне детей, я ..."
24647,Боксёры не занимаются сексом перед боем. Потом...
24648,Судья на заседании так долго призывал к порядк...


In [8]:
# для токенизации используем стеммер Портера
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) \
                   for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-ЯёЁ]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [9]:
# переводим датафрейм в словарь с текстом анекдота и с листом токенов
df_dict_duplicates = dict() 
for i, text in tqdm(df['text'].iteritems()):
    df_dict_duplicates[i] = [text, token_and_stem(text)]

# удаляем из словаря повторяющиеся анекдоты (те у кого один и тот же лист токенов)
temp = set()
df_dict = dict()
for key, value in df_dict_duplicates.items():
    if tuple(set(value[1])) not in temp:
        temp.add(tuple(set(value[1])))
        df_dict[key] = value

print("Удалено",  len(df_dict_duplicates) - len(df_dict), "дубликатов")

15583it [00:45, 339.78it/s]


Удалено 834 дубликатов


In [10]:
def dummy_fun(doc):
    return doc

stopwords = nltk.corpus.stopwords.words('russian')

# расширяем список стоп-слов
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])

# создаем лист с токенами всех анекдотов
tokenized = [value[1] for value in tqdm(df_dict.values())]

max_df = 0.6
min_df = 1e-3
max_features = 100000
ngram_range = (1, 3)
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, max_features=max_features,
                                   min_df=min_df, stop_words=stopwords,
                                   tokenizer=dummy_fun, preprocessor=dummy_fun,
                                   use_idf=True, ngram_range=ngram_range)

tfidf_matrix = tfidf_vectorizer.fit_transform(tokenized)
tfidf_matrix

100%|██████████| 14749/14749 [00:00<00:00, 2965285.67it/s]
C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['б', 'г', 'д', 'е', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'х', 'ц', 'ч', 'ш', 'щ', 'ы', 'ь', 'э', 'ю'] not in stop_words.
  warnings.warn(


<14749x4634 sparse matrix of type '<class 'numpy.float64'>'
	with 329170 stored elements in Compressed Sparse Row format>

In [11]:
# экспериментальном путем выведено, что оптимальное количество кластеров ~200
n_clusters = 200

agglo = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean')
answer = agglo.fit_predict(tfidf_matrix.toarray())
answer

array([  1,   1,   2, ...,   1,  17, 133], dtype=int64)

In [13]:
# добавляем в словарь значения кластеров
for i, (key, value) in enumerate(df_dict.items()):
    df_dict[key].append(answer[i])

# переводим словарь в датафрейм
df = pd.DataFrame.from_dict(df_dict, orient='index')
df.to_csv('clustered_posts.csv')
df

,0,1,2
0,"Стоят два наркомана на перроне, мимо проезжает...","[сто, два, наркома, на, перрон, мим, проезжа, ...",1
1,В советские времена объявили месяц вежливого о...,"[в, советск, врем, объяв, месяц, вежлив, обслу...",1
2,Полицейские повязали целую группу проституток ...,"[полицейск, повяза, цел, групп, проституток, с...",2
5,"— Дед, почему наше село называется Астафьево? ...","[дед, поч, наш, сел, называ, астафьев, дык, на...",79
8,"— Алло, здравствуйте, это Ремонт Ноутбуков? — ...","[алл, здравств, эт, ремонт, ноутбук, да, здрав...",16
...,...,...,...
24622,- Откуда у тебя топор? - Мне Королева эльфов ...,"[откуд, у, теб, топор, мне, королев, эльф, дал...",2
24632,— Официант! Почему у меня кофе воняет хуями? —...,"[официант, поч, у, мен, коф, воня, ху, а, вы, ...",20
24637,— Давай поедем на выходные к океану. — К како...,"[дава, поед, на, выходн, к, океан, к, как, оке...",1
24639,"— Девушка, как вы прекрасны в этом вечернем ту...","[девушк, как, вы, прекрасн, в, эт, вечерн, туа...",17


In [17]:
pd.set_option('display.max_colwidth', 0) 
df[df[2] == 100].head() # анекдоты про Поручика

,0,1,2
74,"Перед званым вечером у Наташи Ростовой Ржевский отчитывает гусаров: не материться, не нажираться, дам прилюдно за задницу не хватать и т. п. Видя скованность гусар за ужином, Наташа задает вопрос: «Господа гусары у меня тут есть 29 подсвечников, а я купила тридцать свечей. Куда бы вставить тридцатую?» Поручик Ржевский, вскакивая из-за стола: ""в пизду ахахаххахахааххахаххахаха""","[перед, зван, вечер, у, наташ, ростов, ржевск, отчитыва, гусар, не, матер, не, нажира, дам, прилюдн, за, задниц, не, хвата, и, т., п., вид, скован, гусар, за, ужин, наташ, зада, вопрос, господ, гусар, у, мен, тут, ест, подсвечник, а, я, куп, тридца, свеч, куд, бы, встав, тридцат, поручик, ржевск, вскакив, из-з, стол, в, пизд, ахахаххахахааххахаххахах]",100
395,"День рождения Наташи Ростовой. Наташа пригласила на праздник поручика Ржевского и всех гусаров полка. За столом Наташа пытается завести светскую беседу: - Вы знаете, я купила 17 свечек для праздничного торта, а на него влезло только 16. Ума не приложу, куда всунуть еще одну?... Гусары кричат хором: - В ПИЗДУ! Ржевский не инструктировал гусаров.","[ден, рожден, наташ, ростов, наташ, приглас, на, праздник, поручик, ржевск, и, всех, гусар, полк, за, стол, наташ, пыта, завест, светск, бесед, вы, знает, я, куп, свечек, для, праздничн, торт, а, на, нег, влезл, тольк, ум, не, прилож, куд, всунут, ещ, одн, гусар, кричат, хор, в, пизд, ржевск, не, инструктирова, гусар]",100
413,"Отечественная война 1812 года, гусары сидят на совете перед Бородинской битвой, курят трубки, обсуждают грядущее сражение. Входит Кутузов. – Господа гусары! До меня дошли слухи, что среди присутствующих завёлся измененник! Этот человек донёс агентам Бонапарта о грядущей расстановке войск, тем самым нарушив присягу Государю, и потому повинен смерти. Однако, нам пока так и не удалось его вычислить. Ржевский! Вы известны своим интеллектом и проницательностью, а также верностью Отечеству. Поручик, не могли бы вы показать нам, кто их присутствующих здесь – предатель? Ржевский, закинув ногу за ногу: – А могу-с.","[отечествен, войн, год, гусар, сид, на, совет, перед, бородинск, битв, кур, трубк, обсужда, грядущ, сражен, вход, кутуз, господ, гусар, до, мен, дошл, слух, что, сред, присутств, завел, измененник, этот, человек, донес, агент, бонапарт, о, грядущ, расстановк, войск, тем, сам, наруш, присяг, государ, и, пот, повин, смерт, однак, нам, пок, так, и, не, уда, ег, вычисл, ржевск, вы, известн, сво, интеллект, и, проницательн, а, такж, верност, отечеств, поручик, не, могл, бы, вы, показа, нам, кто, их, присутств, зде, предател, ржевск, закинув, ног, за, ног, а, могу-с]",100
524,"Поручик Ржевский танцует с дамой на балу. Дама томно поднимает глазки и спрашивает: – Поручик, у меня не слишком глубокое декольте? Поручик заглядывает и говорит: – А у Вас волосы на груди растут? Дама в ужасе: – Нет, что Вы! – Тогда глубокое.","[поручик, ржевск, танц, с, дам, на, бал, дам, томн, поднима, глазк, и, спрашива, поручик, у, мен, не, слишк, глубок, декольт, поручик, заглядыва, и, говор, а, у, вас, волос, на, груд, растут, дам, в, ужас, нет, что, вы, тогд, глубок]",100
1562,"Поручик Ржевский просит своего денщика: — Помоги, любезнейший! Сегодня у Наташи день рождения, я приглашен. Придумай что-нибудь веселенькое, чем бы я мог развеселить и поразить своим искрометным умом гостей! — Пожалуйста... В момент, когда задуют свечи на торте, а свет еще не включат, спросите: «Господа! Темно, как у негра где?» И всем станет весело. Поручик отправился на праздник, дождался нужного момента и, когда Наташа задула свечи, спросил: — Господа! Темно как в жопе у кого?","[поручик, ржевск, прос, сво, денщик, помог, любезн, сегодн, у, наташ, ден, рожден, я, приглаш, придума, что-нибуд, веселеньк, чем, бы, я, мог, развесел, и, пораз, сво, искрометн, ум, гост, пожалуйст, в, момент, когд, зад, свеч, на, торт, а, свет, ещ, не, включат, спрос, господ, темн, как, у, негр, где, и, всем, станет, весел, поручик, отправ, на, праздник, дожда, нужн, момент, и, когд, на

In [33]:
df[df[2] == 157].head() # анекдоты про Буратино

,0,1,2
1576,"Подросший Буратино спрашивает у Папы Карло: — Папа, а почему мне нравится Пьеро, а не Мальвина? — Потому что ты сделан из голубой ели!","[подросш, буратин, спрашива, у, пап, карл, пап, а, поч, мне, нрав, пьер, а, не, мальвин, пот, что, ты, сдела, из, голуб, ел]",157
3241,"Чтобы запастись березовым соком на всю зиму, Папа Карло всю осень давал Буратино мочегонное.","[чтоб, запаст, березов, сок, на, всю, зим, пап, карл, всю, осен, дава, буратин, мочегон]",157
4946,Приходит Буратино на завод. - Вы уголь принимаете? - Да. Буратино садится и начинает дрочить.,"[приход, буратин, на, завод, вы, угол, принима, да, буратин, сад, и, начина, дроч]",157
7316,"— Ты будешь мне опорой, сынок, — сказал папа Карло и переделал Буратино в костыль.","[ты, будеш, мне, опор, сынок, сказа, пап, карл, и, передела, буратин, в, костыл]",157
10631,В полумраке каморки Папа Карло наступает на кучку опилок: - Буратино! Гадина! Опять насрал!,"[в, полумрак, каморк, пап, карл, наступа, на, кучк, опилок, буратин, гадин, опя, насра]",157
